<a href="https://colab.research.google.com/github/Sai1099/App-Analysis/blob/main/bajaj_finserv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install findspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=335ac431e0518f1ba258e201f2cfd859461a7ab7547c1fe25c0f4f4063e48129
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql.types import StructField,StructType,IntegerType,StringType

In [7]:
from pyspark.sql.functions import *

In [77]:
spark = SparkSession.builder.appName("Bajaj-finserv").getOrCreate()

In [9]:
!pip install spacy

In [10]:
import spacy

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
nlp = spacy.load("en_core_web_sm")


In [13]:
excluded_words = spacy.lang.en.stop_words.STOP_WORDS

In [14]:
print(excluded_words)

{'just', 'these', 'give', 'n‘t', 'quite', 'to', 'during', 'fifteen', 'seeming', 'me', 'before', 'bottom', 'from', 'latterly', 'becoming', 'keep', 'throughout', 'in', 'same', 'sixty', 'nor', 'been', 'enough', 're', 'say', 'either', 'whenever', 'the', 'a', 'might', 'my', 'toward', 'across', 'does', 'forty', 'only', 'itself', 'twelve', 'under', 'onto', 'himself', 'any', 'hundred', 'may', 'here', 'once', 'doing', 'since', 'besides', 'two', 'take', 'various', 'part', "'re", 'no', 'around', 'done', 'again', 'own', 'her', 'empty', 'several', 'mostly', 'beside', 'were', 'also', 'though', 'could', 'every', 'n’t', 'meanwhile', 'below', 'as', 'along', 'by', 'even', 'most', 'anywhere', 'front', 'thereupon', 'really', 'few', 'who', 'against', 'four', 'for', 'he', 'upon', 'until', 'beforehand', 'always', 'am', 'whereby', 'nevertheless', 'out', 'go', 'why', 'five', 'hers', "'s", 'make', 'she', 'you', 'both', "'ve", 'together', 'somehow', 'herein', 'through', 'above', 'whom', 'become', 'much', 'using'

In [78]:
schema = StructType([ \
    StructField("Name",StringType(),True), \
    StructField("Rating",IntegerType(),True), \
    StructField("Review",StringType(),True), \
    StructField("Date",StringType(),True), \
    StructField("Useful",IntegerType(),True), \
    StructField("Answered",StringType(),True)
    ])

In [79]:
main_df = spark.read.csv('/content/reviews.txt', schema=schema, sep=']')


In [80]:
main_df = main_df.withColumn("Useful",
                             when(col("Useful").isNull(), 0)
                             .otherwise(col("Useful")))

In [81]:
main_df.show(40)

+--------------------+------+--------------------+------------------+------+--------+
|                Name|Rating|              Review|              Date|Useful|Answered|
+--------------------+------+--------------------+------------------+------+--------+
|        Ayushi Hazra|     5|Very easy to use ...|September 28, 2024|    14|     Yes|
|   Shane Robert Tate|     1|The UI is not use...|September 27, 2024|     3|     Yes|
|      Kunga Wangchuk|     5|Very easy to use ...|    April 27, 2024|     0|     Yes|
|Referral Code - S...|     5|Wonderful app!! I...|September 28, 2024|     4|     Yes|
|Referral Code - D...|     5|This Bajaj Finser...|September 28, 2024|     0|     Yes|
|         Omkar Lodhi|     5|Recently started ...|September 27, 2024|    27|     Yes|
|         MANAV YADAV|     1|This app is a com...|September 28, 2024|     0|     Yes|
|       Rahul Waghela|     1|If I could give 0...| September 5, 2024|   165|     Yes|
|          Abdul Babu|     2|Previously it was...|Sept

In [82]:
main_df.createOrReplaceTempView("bajaj")

In [83]:
cleaned = spark.sql("SELECT Name,Review,Useful FROM bajaj;")

In [84]:
cleaned.show(truncate=False)

+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Name                     |Review                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [85]:
words = main_df.select(explode(split(col("Review")," ")).alias("word"))

In [86]:
words_finder = words.withColumn("words_lower",lower(col("word")))

In [87]:
words_finder = words_finder.groupBy("words_lower").agg(count("words_lower").alias("count")).orderBy(desc("count"))

In [88]:
filtered_words = words_finder.filter(~col("words_lower").isin(excluded_words))

In [89]:
additional_words = ["but", "can't", "iam", "i'm", "very", "this", "it", "it's", "like", "time",
                    "i", "a", "of", "in", "for", "that", "have", "or", "be", "me", "if", "as",
                    'so', "any", "even", "your", 'get', "am", "at", "do", "has","the","to","and","my","is","not","with","from","no","on","when","don't","will","been","an","only","then","also","by","you","are","they","are","was","can","all","use","doesn't","had","than","up",".","we","never","many","every","what","why","while","there","now","just","make","its","try","their","used","after","still"]


In [90]:
filtered_words = words_finder.filter(~col("words_lower").isin(additional_words))

In [91]:
filtered_words.show(60)

+-------------+-----+
|  words_lower|count|
+-------------+-----+
|          app| 1044|
|        bajaj|  365|
|         easy|  352|
|         loan|  351|
|         card|  296|
|          emi|  280|
|     customer|  227|
|   experience|  194|
|    interface|  178|
|         app.|  165|
|         user|  160|
|    financial|  159|
|         good|  158|
|        worst|  155|
|  application|  154|
|          one|  151|
|      finserv|  149|
|        using|  143|
|      service|  140|
|          upi|  134|
|      payment|  120|
|        loans|  119|
|       please|  103|
|      finance|   98|
|        money|   97|
|       really|   96|
|     features|   96|
|     managing|   95|
|        great|   94|
|      process|   92|
|       offers|   90|
|      without|   89|
|       credit|   86|
|     friendly|   85|
|          bad|   84|
|      support|   83|
|       smooth|   82|
|          pay|   82|
|     payments|   80|
|         much|   80|
|      showing|   78|
|       update|   78|
|        w

In [92]:
mainde = spark.sql("SELECT * FROM bajaj WHERE Rating < 4 AND (Review LIKE '%worst%');")

In [93]:
mainde.select(count("Review")).show()

+-------------+
|count(Review)|
+-------------+
|           79|
+-------------+



### END



In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="siebert/sentiment-roberta-large-english")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

In [ ]:
inputs = tokenizer("Hello, you are waste", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.logits

In [ ]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.2294,  2.4462]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
